<a href="https://colab.research.google.com/github/Geethss/Precog-recruitment-task/blob/main/Word_and_Phrase_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
simlex_df = pd.read_csv('/content/SimLex-999.csv')

In [ ]:
import gensim.downloader as api

word2vec_model = api.load('word2vec-google-news-300')
word2vec_model.save_word2vec_format('/content/GoogleNews-vectors-negative300.bin', binary=True)

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
#                                                                                  1.a) Word Similarity with constraints
from gensim.models import KeyedVectors
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from scipy.stats import pearsonr, spearmanr

import nltk

nltk.download('punkt')
nltk.download('stopwords')

word2vec_model_path = '/content/GoogleNews-vectors-negative300.bin'
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=True)

simlex_df = pd.read_csv('/content/SimLex-999.csv',delimiter='\t', skiprows=1, names=['word1', 'word2', 'POS', 'SimLex999', 'conc(w1)', 'conc(w2)', 'concQ', 'Assoc(USF)', 'SimAssoc333', 'SD(SimLex)'])

wsj_sentences = nltk.corpus.treebank.sents('wsj_0001.mrg')

#flattening sentences into list of words
corpus_tokens = [word.lower() for sentence in wsj_sentences for word in sentence if word.isalpha()]

#Function to calculate average similarity between two lists of words using Word2Vec embeddings
def calculate_similarity(words1, words2):
    try:
        similarities = [word2vec_model.similarity(w1, w2) for w1 in words1 for w2 in words2]
        if similarities:
            avg_similarity = sum(similarities) / len(similarities)
            return avg_similarity
        else:
            return 0.0
    except KeyError:
        return 0.0

def preprocess_and_calculate_similarity(row):
    word1 = row['word1'].lower()
    word2 = row['word2'].lower()

    #Tokenizing and filtering stopwords to caluclate similarity score
    tokens1 = [word for word in word_tokenize(word1) if word.isalpha() and word not in stopwords.words('english')]
    tokens2 = [word for word in word_tokenize(word2) if word.isalpha() and word not in stopwords.words('english')]


    similarity_score = calculate_similarity(tokens1, tokens2)
    return similarity_score

#final result - accuracy caluclation
simlex_df['predicted_similarity'] = simlex_df.apply(preprocess_and_calculate_similarity, axis=1)

pearson_corr, _ = pearsonr(simlex_df['SimLex999'], simlex_df['predicted_similarity'])
spearman_corr, _ = spearmanr(simlex_df['SimLex999'], simlex_df['predicted_similarity'])

print(f"Pearson correlation: {pearson_corr}")
print(f"Spearman correlation: {spearman_corr}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Pearson correlation: 0.4547206952123591
Spearman correlation: 0.4428068488571049


In [ ]:
#                                                                                    Word similarity without any constraints

#Note: Recieved an error while executing this code which I had talked about in the second takeaway
from gensim.models import KeyedVectors
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import brown
from itertools import product
from scipy.stats import pearsonr, spearmanr
import numpy as np

import nltk
nltk.download('brown')
nltk.download('punkt')

word2vec_model_path = '/content/GoogleNews-vectors-negative300.bin'
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=True)

brown_corpus_words = set(brown.words())

def calculate_similarity(words1, words2):
    try:
        vectors1 = np.array([word2vec_model[word] for word in words1 if word in word2vec_model])
        vectors2 = np.array([word2vec_model[word] for word in words2 if word in word2vec_model])

        if len(vectors1) == 0 or len(vectors2) == 0:
            return 0.0

        avg_similarity = np.dot(vectors1.mean(axis=0), vectors2.mean(axis=0))
        return avg_similarity

    except KeyError as e:
        print(f"KeyError: {e}, words1: {words1}, words2: {words2}")
        return 0.0

def preprocess_and_calculate_similarity(row):
    word1 = row['word1'].lower()
    word2 = row['word2'].lower()

    #Tokenizing and filtering stop words
    tokens1 = [word for word in word_tokenize(word1) if word.isalpha() and word not in brown_corpus_words]
    tokens2 = [word for word in word_tokenize(word2) if word.isalpha() and word not in brown_corpus_words]

    similarity_score = calculate_similarity(tokens1, tokens2)
    return similarity_score

simlex_df = pd.read_csv('/content/SimLex-999.csv', delimiter='\t', skiprows=1, names=['word1', 'word2', 'POS', 'SimLex999', 'conc(w1)', 'conc(w2)', 'concQ', 'Assoc(USF)', 'SimAssoc333', 'SD(SimLex)'])
simlex_df['predicted_similarity'] = simlex_df.apply(preprocess_and_calculate_similarity, axis=1)

#to handle the NaN values present in the dataset
simlex_df['predicted_similarity'].fillna(0.0, inplace=True)

if simlex_df['SimLex999'].nunique() == 1:
    print("Warning: 'SimLex999' column is constant, correlation coefficients are not defined.")
else:
    simlex_df.dropna(subset=['SimLex999'], inplace=True)

    pearson_corr, _ = pearsonr(simlex_df['SimLex999'], simlex_df['predicted_similarity'])
    spearman_corr, _ = spearmanr(simlex_df['SimLex999'], simlex_df['predicted_similarity'])

    print(f"Pearson correlation: {pearson_corr}")
    print(f"Spearman correlation: {spearman_corr}")





[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Pearson correlation: nan
Spearman correlation: nan


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


In [ ]:
#                                                                    Working code of Word similarity without constraints
from gensim.models import KeyedVectors, Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import brown
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

import nltk
nltk.download('brown')
nltk.download('punkt')

word2vec_model_path = '/content/GoogleNews-vectors-negative300.bin'
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=True)

brown_corpus_words = set(brown.words())

#Function to calculate similarity
def calculate_similarity(words1, words2):
    try:
        vectors1 = np.array([word2vec_model[word] for word in words1 if word in word2vec_model])
        vectors2 = np.array([word2vec_model[word] for word in words2 if word in word2vec_model])

        if len(vectors1) == 0 or len(vectors2) == 0:
            return 0.0

        avg_similarity = np.dot(vectors1.mean(axis=0), vectors2.mean(axis=0))
        return avg_similarity

    except KeyError as e:
        print(f"KeyError: {e}, words1: {words1}, words2: {words2}")
        return 0.0

#Function to preprocess and calculate similarity
def preprocess_and_calculate_similarity(row):
    word1 = row['word1'].lower()
    word2 = row['word2'].lower()

    #Tokenizing and filtering stopwords using Brown Corpus
    tokens1 = [word for word in word_tokenize(word1) if word.isalpha() and word not in brown_corpus_words]
    tokens2 = [word for word in word_tokenize(word2) if word.isalpha() and word not in brown_corpus_words]

    similarity_score = calculate_similarity(tokens1, tokens2)
    return similarity_score

simlex_df = pd.read_csv('/content/SimLex-999.csv', delimiter='\t', skiprows=1, names=['word1', 'word2', 'POS', 'SimLex999', 'conc(w1)', 'conc(w2)', 'concQ', 'Assoc(USF)', 'SimAssoc333', 'SD(SimLex)'])

simlex_df['predicted_similarity'] = simlex_df.apply(preprocess_and_calculate_similarity, axis=1)
simlex_df['predicted_similarity'].fillna(0.0, inplace=True)
sentences = [word_tokenize(sentence.lower()) for sentence in simlex_df['word1']] + [word_tokenize(sentence.lower()) for sentence in simlex_df['word2']]
word2vec_model_tuned = Word2Vec(sentences, vector_size=300, window=10, min_count=5, workers=4, epochs=20)

simlex_df['predicted_similarity_tuned'] = simlex_df.apply(preprocess_and_calculate_similarity, axis=1)
mse_tuned = mean_squared_error(simlex_df['SimLex999'], simlex_df['predicted_similarity_tuned'])
print(f"Tuned Mean Squared Error: {mse_tuned}")


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tuned Mean Squared Error: 27.637556656656656


Phrase Similarity


In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("PiC/phrase_similarity")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import nltk
from datasets import load_dataset

nltk.download('punkt')

word2vec_model_path = '/content/GoogleNews-vectors-negative300.bin'
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=True)

dataset = load_dataset("PiC/phrase_similarity")

train_data = pd.DataFrame(dataset["train"])
validation_data = pd.DataFrame(dataset["validation"])
test_data = pd.DataFrame(dataset["test"])

#Function to calculate similarity between two lists of words using Word2Vec embeddings
def calculate_similarity(words1, words2):
    try:
        similarities = [word2vec_model.similarity(w1, w2) for w1 in words1 for w2 in words2]
        if similarities:
            avg_similarity = sum(similarities) / len(similarities)
            return avg_similarity
        else:
            return 0.0
    except KeyError:
        return 0.0

#Function to aggregate word representations into a phrase representation
def aggregate_word_representations(words):
    word_vectors = [word2vec_model[word] for word in words if word in word2vec_model]
    if word_vectors:
        return sum(word_vectors) / len(word_vectors)
    else:
        return np.zeros(word2vec_model.vector_size)

#Function to calculate similarity for phrases along with the similarity score based on aggregrated word representations
def calculate_phrase_similarity(phrase1, phrase2):
    tokens1 = [word.lower() for word in word_tokenize(phrase1) if word.isalpha()]
    tokens2 = [word.lower() for word in word_tokenize(phrase2) if word.isalpha()]

    phrase_representation1 = aggregate_word_representations(tokens1)
    phrase_representation2 = aggregate_word_representations(tokens2)

    similarity_score = calculate_similarity([phrase_representation1], [phrase_representation2])
    return similarity_score

#Training a Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=1000, random_state=42)
rf_model.fit(X_train, y_train)

predictions = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
}


rf_model = RandomForestClassifier(random_state=42)

#grid search
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search.best_params_)

best_rf_model = grid_search.best_estimator_
predictions = best_rf_model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy: 0.5
Best Hyperparameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
Accuracy: 0.5


In [ ]:
#tried to improve the accuracy - used this code to know where I wen wrong
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
from datasets import load_dataset


nltk.download('punkt')

word2vec_model_path = '/content/GoogleNews-vectors-negative300.bin'
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=True)

dataset = load_dataset("PiC/phrase_similarity")

train_data = pd.DataFrame(dataset["train"])
validation_data = pd.DataFrame(dataset["validation"])
test_data = pd.DataFrame(dataset["test"])

print("Missing Values in Train Set:")
print(train_data.isnull().sum())

print("\nMissing Values in Validation Set:")
print(validation_data.isnull().sum())

print("\nMissing Values in Test Set:")
print(test_data.isnull().sum())

#to handle missing values
train_data.fillna(0, inplace=True)
validation_data.fillna(0, inplace=True)
test_data.fillna(0, inplace=True)

#to detect outliers and handle them
from scipy.stats import zscore
z_scores = zscore(train_data[['label']])
outliers = (np.abs(z_scores) > 3).all(axis=1)

print("\nNumber of Outliers in Train Set:", outliers.sum())
train_data = train_data[~outliers]


#function to calculate similarity between two lists of words using Word2Vec embeddings
def calculate_similarity(words1, words2):
    try:
        similarities = [word2vec_model.similarity(w1, w2) for w1 in words1 for w2 in words2]
        if similarities:
            avg_similarity = sum(similarities) / len(similarities)
            return avg_similarity
        else:
            return 0.0
    except KeyError:
        return 0.0

#function to aggregate word representations into a phrase representation
def aggregate_word_representations(words):
    word_vectors = [word2vec_model[word] for word in words if word in word2vec_model]
    if word_vectors:
        return sum(word_vectors) / len(word_vectors)
    else:
        return np.zeros(word2vec_model.vector_size)

#function to calculate similarity for phrases
def calculate_phrase_similarity(phrase1, phrase2):
    tokens1 = [word.lower() for word in word_tokenize(phrase1) if word.isalpha()]
    tokens2 = [word.lower() for word in word_tokenize(phrase2) if word.isalpha()]

    phrase_representation1 = aggregate_word_representations(tokens1)
    phrase_representation2 = aggregate_word_representations(tokens2)

    similarity_score = calculate_similarity([phrase_representation1], [phrase_representation2])
    return similarity_score

#converting datasets to pytorch tensors
X_train = torch.tensor(train_data.apply(lambda row: calculate_phrase_similarity(row['phrase1'], row['phrase2']), axis=1).to_numpy().reshape(-1, 1), dtype=torch.float32)
y_train = torch.tensor(train_data['label'].to_numpy(), dtype=torch.float32)

X_val = torch.tensor(validation_data.apply(lambda row: calculate_phrase_similarity(row['phrase1'], row['phrase2']), axis=1).to_numpy().reshape(-1, 1), dtype=torch.float32)
y_val = torch.tensor(validation_data['label'].to_numpy(), dtype=torch.float32)

X_test = torch.tensor(test_data.apply(lambda row: calculate_phrase_similarity(row['phrase1'], row['phrase2']), axis=1).to_numpy().reshape(-1, 1), dtype=torch.float32)
y_test = torch.tensor(test_data['label'].to_numpy(), dtype=torch.float32)

class PhraseSimilarityNN(nn.Module):
    def __init__(self):
        super(PhraseSimilarityNN, self).__init__()
        self.fc = nn.Linear(1, 1)

    def forward(self, x):
        x = self.fc(x)
        return x

model = PhraseSimilarityNN()

#Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    #Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))

    #Backward pass and optimization
    loss.backward()
    optimizer.step()

    #Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val.view(-1, 1))
        val_predictions = torch.round(torch.sigmoid(val_outputs))
        val_accuracy = accuracy_score(y_val.numpy(), val_predictions.numpy())

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}")

#testing the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_predictions = torch.round(torch.sigmoid(test_outputs))
    test_accuracy = accuracy_score(y_test.numpy(), test_predictions.numpy())

print(f"Test Accuracy: {test_accuracy:.4f}")




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Missing Values in Train Set:
phrase1      0
phrase2      0
sentence1    0
sentence2    0
label        0
idx          0
dtype: int64

Missing Values in Validation Set:
phrase1      0
phrase2      0
sentence1    0
sentence2    0
label        0
idx          0
dtype: int64

Missing Values in Test Set:
phrase1      0
phrase2      0
sentence1    0
sentence2    0
label        0
idx          0
dtype: int64

Number of Outliers in Train Set: 0
Epoch [1/10], Loss: 0.7568, Val Loss: 0.7566, Val Accuracy: 0.5000
Epoch [2/10], Loss: 0.7566, Val Loss: 0.7565, Val Accuracy: 0.5000
Epoch [3/10], Loss: 0.7565, Val Loss: 0.7563, Val Accuracy: 0.5000
Epoch [4/10], Loss: 0.7563, Val Loss: 0.7561, Val Accuracy: 0.5000
Epoch [5/10], Loss: 0.7561, Val Loss: 0.7559, Val Accuracy: 0.5000
Epoch [6/10], Loss: 0.7559, Val Loss: 0.7558, Val Accuracy: 0.5000
Epoch [7/10], Loss: 0.7558, Val Loss: 0.7556, Val Accuracy: 0.5000
Epoch [8/10], Loss: 0.7556, Val Loss: 0.7554, Val Accuracy: 0.5000
Epoch [9/10], Loss: 0.7554